**Introduction:**

Hacker News is a popular site among the technology and startup circles. Users can either ask or show news on their posts. Similar to Reddit, the posts can be voted or commented upon.

Goal of this project is to determine two things:
1. Do Ask HN or Show HN receive more comments on average?
2. Do posts created at a certain time receive more comments on average?


In [1]:
from csv import reader

In [2]:
data = open('hacker_news.csv')
data_read = reader(data)

hn = list(data_read)

In [3]:
# print first 5 rows
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [4]:
# extract headers
headers = hn[0]

# remove headers from hn
hn = hn[1:]

In [5]:
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [6]:
print(hn[:5])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


***Separate the posts into three categories:***
1. Ask HN
2. Show HN
3. All remaining posts

In [7]:
ask_posts = []
show_posts = []
other_posts = []

In [8]:
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

In [9]:
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


***Determine if ask posts or show posts recieve more comments on average.***

In [10]:
total_ask_comments = 0
total_show_comments = 0

In [11]:
for row in ask_posts:
    total_ask_comments += int(row[4])

avg_ask_comments = total_ask_comments / len(ask_posts)

In [12]:
print(avg_ask_comments)

14.038417431192661


In [13]:
for row in show_posts:
    total_show_comments += int(row[4])
    
avg_show_comments = total_show_comments / len(show_posts)

In [14]:
print(avg_show_comments)

10.31669535283993


Seems like Ask Posts, on average, receive more comments than Show Posts.

***Are Ask Posts more likely to attract comments at a certain time?***

Approach:
1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts recieve by hour created.

In [15]:
import datetime as dt

In [16]:
result_list = []

In [19]:
for row in ask_posts:
    created = row[6]
    num_comments = int(row[4])
    result_list.append([created, num_comments])

In [22]:
counts_by_hour = {} # contains the number of ask posts created during each hour of the day. 
comments_by_hour = {} # contains the corresponding number of comments ask posts created at each hour received.

In [25]:
for row in result_list:
    created_date = dt.datetime.strptime(row[0], '%m/%d/%Y %H:%M')
    hour = created_date.strftime('%H')
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

In [45]:
# calculate the number of comments per post for any given hour
avg_by_hour = []

for row in counts_by_hour:
    hour = row
    avg = comments_by_hour[hour] / counts_by_hour[hour]
    avg_by_hour.append([hour, avg])

In [46]:
print(avg_by_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


In [47]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

In [48]:
print(swap_avg_by_hour)

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


In [50]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)

In [64]:
print('Top 5 Hours for Ask Posts Comments')

for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(row[1], '%H')
    hour = hour.strftime('%H')
    average = row[0]
    print('{}:00: {:.2f} average comments per post'.format(hour, average))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


Posting during the following times would net the highest chance to receive comments:
1. 3pm EST or 12pm PST
2. 2am EST or 11pm PST
3. 8pm EST or 5pm PST